In [1]:
%fs ls /FileStore/tables/

In [2]:
HR_data = spark.read.csv('dbfs:/FileStore/tables/HR_comma_sep.csv',inferSchema=True,header=True)

In [3]:
HR_data.head()

In [4]:
HR_data.printSchema()

In [5]:
HR_data[['dept']].distinct().collect()

In [6]:
#renamed the column to new name
HR_data = HR_data.withColumnRenamed('sales','dept')

In [7]:
HR_data[['dept']].show(2) #if we are give one column name we shd need to give double [[]]. so that it'll be a df. otherwise it'll be column operation

In [8]:
HR_data.describe().show()

In [9]:
###Featurization converting the stirng data to numbers
#StringIndexer is converting the string to numbeArs
import pyspark.ml.feature as ft
transformer_dept=ft.StringIndexer(inputCol='dept',outputCol='dept_en')
transformer_salary=ft.StringIndexer(inputCol='salary',outputCol='salary_en')


In [10]:
#Convert into numbers into verctors==output column name should be features
Featurecreate=ft.VectorAssembler(inputCols=[ 'satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident',  'promotion_last_5years','dept_en','salary_en'],outputCol='features')

In [11]:
HR_data.columns

In [12]:
import pyspark.ml.classification as cl

In [13]:
#Creating estimator
logistic = cl.LogisticRegression(maxIter=10,regParam=0.01,labelCol='left')

In [14]:
from pyspark.ml import Pipeline

In [15]:
#Create pipeline connecting transformer and estimator
pipeline=Pipeline(stages=[transformer_dept,transformer_salary,Featurecreate,logistic])

In [16]:
#Split data for test and train
#seed control random data generated
hr_data_train,hr_data_test = HR_data.randomSplit([0.7,0.3],seed=100)

In [17]:
model=pipeline.fit(hr_data_train)

In [18]:
test_out=model.transform(hr_data_test)

In [19]:
test_out

In [20]:
test_out[['probability']].take(2)

In [21]:
import pyspark.ml.evaluation as ev

In [22]:
evaluator=ev.BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='left')

In [23]:
evaluator.evaluate(test_out)

In [24]:
#Convert the fields type into double
from pyspark.sql.functions import *

hr_data = HR_data.withColumn('left', col('left').cast('double'))

In [25]:
hr_data.printSchema()

In [26]:

## Create random forest estimator
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="left", featuresCol="features", 
                            numTrees=10, maxDepth=5.0)

In [27]:
pipeline=Pipeline(stages=[transformer_dept,transformer_salary,Featurecreate,rf])

In [28]:
#Split data for test & train
#seed controls the random data generated
hr_data_train, hr_data_test = hr_data.randomSplit([0.7,0.3],seed=100)

In [29]:
model = pipeline.fit(hr_data_train)

In [30]:

test_out = model.transform(hr_data_test)

In [31]:
test_out.show()

In [32]:
#BinaryClassification
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', 
                                             labelCol='left')

In [33]:
evaluator.evaluate(test_out)

In [34]:
from pyspark.ml.classification import GBTClassifier

In [35]:
gbt = GBTClassifier(labelCol='left', featuresCol='features' , maxIter=10)

In [36]:
pipeline=Pipeline(stages=[transformer_dept,transformer_salary,Featurecreate,gbt])

In [37]:
#Split data for test & train
#seed controls the random data generated
hr_data_train, hr_data_test = hr_data.randomSplit([0.7,0.3],seed=100)

In [38]:
model = pipeline.fit(hr_data_train)

In [39]:
test_out = model.transform(hr_data_test)

In [40]:
test_out.show()

In [41]:
#BinaryClassification
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', 
                                             labelCol='left')

In [42]:
evaluator.evaluate(test_out)